In [1]:
import polars as pl
import polars_qt as pqt

In [2]:
df = pl.DataFrame({"a": pl.Series([1, 2, 3]).qt.with_unit([("m", (1, 1))])})
df

a
struct[2]
"{1,[{""m"",{1,1}}]}"
"{2,[{""m"",{1,1}}]}"
"{3,[{""m"",{1,1}}]}"


In [3]:
df.dtypes

[Struct({'value': Int64, 'unit': List(Struct({'name': String, 'power': Struct({'numer': Int64, 'denom': Int64})}))})]

In [4]:
df.select(pl.col("a").qt.noop())

input: shape: (3,)
Series: 'a' [struct[2]]
[
	{1,[{"m",{1,1}}]}
	{2,[{"m",{1,1}}]}
	{3,[{"m",{1,1}}]}
]
result: shape: (3,)
Series: 'value' [i64]
[
	1
	2
	3
]
unit: shape: (3,)
Series: 'unit' [list[struct[2]]]
[
	[{"m",{1,1}}]
	[{"m",{1,1}}]


a
struct[2]
"{1,[{""m"",{1,1}}]}"
"{2,[{""m"",{1,1}}]}"
"{3,[{""m"",{1,1}}]}"


	[{"m",{1,1}}]
]
add_unit: unit_val: Scalar { dtype: List(Struct([Field { name: "name", dtype: String }, Field { name: "power", dtype: Struct([Field { name: "numer", dtype: Int64 }, Field { name: "denom", dtype: Int64 }]) }])), value: List(shape: (1,)
Series: '' [struct[2]]
[
	{"m",{1,1}}
]) }


In [48]:
df['a']

{'value': 1, 'unit': [{'name': 'm', 'power': {'numer': 1, 'denom': 1}}]}

Numeric cast in polars

In [5]:
start = pl.DataFrame(
    {
        "int": [1, 2, 3],
        "float": [1.0, 2.0, 3.0],
    }
)
sum = start.with_columns(sum=pl.col("int") + pl.col("float"))
start.dtypes, sum.dtypes

([Int64, Float64], [Int64, Float64, Float64])

In [6]:
series = pl.Series([1, 2, 3, 4, 5])
dtype = pl.Struct({"value": series.dtype, "unit": pl.String})

In [7]:
pl.struct(value=series, unit=pl.lit("hello"), eager=True).dtype

Struct({'value': Int64, 'unit': String})

## Pint

In [4]:
import pint

In [9]:
ureg = pint.UnitRegistry()

In [10]:
(1 * ureg.meter / ureg.second).to("km/h").units._units.unit_items()

dict_items([('kilometer', 1), ('hour', -1)])

In [35]:
frac_unit = (2 * ureg.meter) ** (1/2) * ureg.second

In [38]:
frac_unit.units._units?

Type:        UnitsContainer
String form: meter ** 0.5 * second
Length:      2
File:        ~/development/polars-qt/.pixi/envs/default/lib/python3.13/site-packages/pint/util.py
Docstring:  
The UnitsContainer stores the product of units and their respective
exponent and implements the corresponding operations.

UnitsContainer is a read-only mapping. All operations (even in place ones)
return new instances.

Parameters
----------
non_int_type
    Numerical type used for non integer values.

In [30]:
from polars_qt.pint import pint_to_pqt, pqt_to_pint, QtQuantity, QtUnit
from dataclasses import asdict

In [31]:
qt_unit = pint_to_pqt(ureg.meter)

In [39]:
def pqt_to_pint(units: list[QtUnit]) -> pint.Unit:
    """
    Convert a list of QtUnit to a pint unit
    """
    return ureg.Unit(pint.util.UnitsContainer({unit.name: unit.power for unit in units}))

In [40]:
pqt_to_pint(qt_unit)

<Unit('meter')>

In [45]:
asdict(qt_unit[0])

{'name': 'meter', 'power': Fraction(1, 1)}

# Parse pint units

```json
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "Units and Dimensions Schema",
  "type": "object",
  "properties": {
    "units": {
      "type": "object",
      "patternProperties": {
        "^[a-zA-Z_]+$": {
          "type": "object",
          "properties": {
            "relation": {
              "type": "object",
              "properties": {
                "operation": { "type": "string" },
                "left": { "type": "string" },
                "right": { "type": "string" },
                "base": { "type": "string" },
                "exponent": { "type": "number" },
                "numerator": { "type": ["string", "number"] },
                "denominator": { "type": ["string", "number"] }
              },
              "required": ["operation"]
            },
            "symbol": { "type": "string" },
            "aliases": {
              "type": "array",
              "items": { "type": "string" }
            }
          },
          "required": ["relation"]
        }
      },
      "additionalProperties": false
    },
    "prefixes": {
      "type": "object",
      "patternProperties": {
        "^[a-zA-Z_]+-$": {
          "type": "object",
          "properties": {
            "amount": { "type": "string" },
            "symbol": { "type": "string" },
            "aliases": {
              "type": "array",
              "items": { "type": "string" }
            }
          },
          "required": ["amount"]
        }
      },
      "additionalProperties": false
    },
    "derived_dimensions": {
      "type": "object",
      "patternProperties": {
        "^[a-zA-Z_]+$": {
          "type": "object",
          "properties": {
            "operation": { "type": "string" },
            "left": { "type": "string" },
            "right": { "type": "string" }
          },
          "required": ["operation", "left", "right"]
        }
      },
      "additionalProperties": false
    },
    "aliases": {
      "type": "object",
      "patternProperties": {
        "^[a-zA-Z_]+$": {
          "type": "array",
          "items": { "type": "string" }
        }
      },
      "additionalProperties": false
    },
    "defaults": {
      "type": "object",
      "properties": {
        "group": { "type": "string" },
        "system": { "type": "string" }
      },
      "required": ["group", "system"]
    }
  },
  "required": ["units", "prefixes", "derived_dimensions", "aliases", "defaults"]
}
```